# 環境設定

In [1]:
# 安裝必要套件
!pip install snownlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.6/37.6 MB 14.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for snownlp: filename=snownlp-0.12.3-py3-none-any.whl size=37760946 sha256=f46d8143992071aaee5117d085541d9b0be5cf0f0ea2871e34bcbd2d1145394f
  Stored in directory: /root/.cache/pip/wheels/43/f3/70/8990fc249efeb396007766676706f71dd3d1ca3c023ce522ce
Successfully built snownlp


In [2]:
# 下載資料集檔案
import os
Dataset_File = "Comments_Hotels.xlsx"

if not os.path.isfile(Dataset_File):
  os.system("wget https://raw.githubusercontent.com/cnchi/datasets/master/" + Dataset_File)

In [3]:
# 下載講師自製的 HappyML
import os

if not os.path.isdir("HappyML"):
  os.system("git clone https://github.com/cnchi/HappyML.git")

# 資料前處理

## 載入資料集

In [4]:
# 載入資料集
import pandas as pd
dataset = pd.read_excel(Dataset_File)

# 印出前幾筆做為驗證
dataset.head()

,label,review
0,1,"距離川沙公路較近,但是公車指示不對,如果是""蔡陸線""的話,會非常麻煩.建議用別的路線.房間較..."
1,1,商務大床房，房間很大，床有2M寬，整體感覺經濟實惠不錯!
2,1,早餐太差，無論去多少人，那邊也不加食品的。酒店應該重視一下這個問題了。房間本身很好。
3,1,賓館在小街道上，不大好找，但還好北京熱心同胞很多~賓館設施跟介紹的差不多，房間很小，確實挺小...
4,1,"CBD中心,周圍沒什麼店鋪,說5星有點勉強.不知道為什麼洗手間沒有吹風機"


## 切分自變數與應變數

In [5]:
# 切分自變數 X 與應變數 Y
import HappyML.preprocessor as pp

X, Y = pp.decomposition(dataset, x_columns=[1], y_columns=[0])

In [6]:
X, Y

(                                                 review
 0     距離川沙公路較近,但是公車指示不對,如果是"蔡陸線"的話,會非常麻煩.建議用別的路線.房間較...
 1                          商務大床房，房間很大，床有2M寬，整體感覺經濟實惠不錯!
 2            早餐太差，無論去多少人，那邊也不加食品的。酒店應該重視一下這個問題了。房間本身很好。
 3     賓館在小街道上，不大好找，但還好北京熱心同胞很多~賓館設施跟介紹的差不多，房間很小，確實挺小...
 4                  CBD中心,周圍沒什麼店鋪,說5星有點勉強.不知道為什麼洗手間沒有吹風機
 ...                                                 ...
 7760  尼斯酒店的幾大特點：噪音大、環境差、配置低、服務效率低。如：1、隔壁歌廳的聲音鬧至午夜3點許...
 7761  鹽城來了很多次，第一次住鹽阜賓館，我的確很失望整個牆壁黑咕隆咚的，好像被煙燻過一樣家具非常的...
 7762  看照片覺得還挺不錯的，又是4星級的，但入住以後除了後悔沒有別的，房間挺大但空空的，早餐是有但...
 7763  我們去鹽城的時候那裡的最低氣溫只有4度，晚上冷得要死，居然還不開空調，投訴到酒店客房部，得到...
 7764  說實在的我很失望，之前看了其他人的評論後覺得還可以才去的，結果讓我們大跌眼鏡。我想這家酒店以...
 
 [7765 rows x 1 columns],
       label
 0         1
 1         1
 2         1
 3         1
 4         1
 ...     ...
 7760      0
 7761      0
 7762      0
 7763      0
 7764      0
 
 [7765 rows x 1 columns])

# 以 SnowNLP 預測

In [7]:
# 引入 SnowNLP
from snownlp import SnowNLP

# 建立一個空的串列，儲存預測結果
snownlp_results = []

# 將自變數 X 中的每一條評論，迭代出來
for review in X['review']:
  # 使用 SnowNLP 進行情感分析
  analysis_result = SnowNLP(review)
  # 將結果轉換成 0 或 1
  score = 1 if analysis_result.sentiments > 0.5 else 0
  # 將結果添加到串列中
  snownlp_results.append(score)

In [8]:
# 將真實值與預測值合併成一個 DataFrame 觀察之
import pandas as pd

results = pd.DataFrame({'Y_real': Y.values.ravel(), 'Y_pred': snownlp_results})
results.head()

,Y_real,Y_pred
0,1,0
1,1,0
2,1,0
3,1,1
4,1,0


# 計算模型效能

In [9]:
# 使用 HappyML 計算模型效能
from HappyML.performance import ClassificationPerformance

pfm = ClassificationPerformance(results['Y_real'], results['Y_pred'])

print("Confusion Matrix:\n", pfm.confusion_matrix())
print(f"Accuracy: {pfm.accuracy():.2%}")
print(f"Recall: {pfm.recall():.2%}")
print(f"Precision: {pfm.precision():.2%}")
print(f"F1 Score: {pfm.f_score():.2%}")

Confusion Matrix:
 [[2421   22]
 [4489  833]]
Accuracy: 41.91%
Recall: 57.38%
Precision: 66.23%
F1 Score: 39.37%
